# IoT Anomaly Detection - Data Exploration

This notebook explores the SMAP (Soil Moisture Active Passive) and MSL (Mars Science Laboratory) datasets for anomaly detection.

## Objectives:
1. Load and understand the data structure
2. Analyze statistical properties
3. Visualize time series patterns
4. Identify anomaly characteristics
5. Explore sensor correlations
6. Prepare insights for model development

## 1. Setup and Imports

In [ ]:
# Standard libraries
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Data manipulation
import numpy as np
import pandas as pd
from scipy import stats
from scipy.signal import find_peaks
from scipy.fft import fft, fftfreq

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Machine learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

# Project modules
from src.data_ingestion.data_loader import DataLoader
from src.preprocessing.data_preprocessor import DataPreprocessor
from config.settings import Config

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Libraries imported successfully!")

## 2. Load Configuration and Data

In [ ]:
# Load configuration
config = Config()

# Initialize data loader
data_loader = DataLoader(config)

# Define data paths
SMAP_PATH = os.path.join(config.paths['smap_data'])
MSL_PATH = os.path.join(config.paths['msl_data'])

print(f"SMAP Data Path: {SMAP_PATH}")
print(f"MSL Data Path: {MSL_PATH}")
print(f"\nConfiguration loaded from: {config.config_file}")

In [ ]:
# Load SMAP data
print("Loading SMAP dataset...")
smap_train, smap_test, smap_labels = data_loader.load_smap_data()

print(f"\nSMAP Dataset Shape:")
print(f"  Training: {smap_train.shape if smap_train is not None else 'No data'}")
print(f"  Testing: {smap_test.shape if smap_test is not None else 'No data'}")
print(f"  Labels: {smap_labels.shape if smap_labels is not None else 'No labels'}")

# Load MSL data
print("\nLoading MSL dataset...")
msl_train, msl_test, msl_labels = data_loader.load_msl_data()

print(f"\nMSL Dataset Shape:")
print(f"  Training: {msl_train.shape if msl_train is not None else 'No data'}")
print(f"  Testing: {msl_test.shape if msl_test is not None else 'No data'}")
print(f"  Labels: {msl_labels.shape if msl_labels is not None else 'No labels'}")

## 3. Basic Data Statistics

In [ ]:
def calculate_statistics(data, name):
    """Calculate basic statistics for the dataset"""
    if data is None:
        print(f"No data available for {name}")
        return None
    
    # Convert to DataFrame for easier analysis
    if len(data.shape) == 3:
        # Reshape 3D to 2D (samples * timesteps, features)
        data_2d = data.reshape(-1, data.shape[-1])
    else:
        data_2d = data
    
    df = pd.DataFrame(data_2d)
    
    stats = {
        'Dataset': name,
        'Shape': data.shape,
        'Total Points': np.prod(data.shape),
        'Memory (MB)': data.nbytes / (1024 * 1024),
        'Data Type': data.dtype,
        'Missing Values': np.isnan(data).sum(),
        'Missing %': (np.isnan(data).sum() / np.prod(data.shape)) * 100,
        'Infinity Values': np.isinf(data).sum(),
    }
    
    # Statistical measures
    stats_df = pd.DataFrame({
        'Mean': df.mean(),
        'Std': df.std(),
        'Min': df.min(),
        '25%': df.quantile(0.25),
        'Median': df.median(),
        '75%': df.quantile(0.75),
        'Max': df.max(),
        'Skewness': df.skew(),
        'Kurtosis': df.kurtosis()
    })
    
    return stats, stats_df

# Calculate statistics for all datasets
datasets = [
    (smap_train, 'SMAP Train'),
    (smap_test, 'SMAP Test'),
    (msl_train, 'MSL Train'),
    (msl_test, 'MSL Test')
]

all_stats = []
for data, name in datasets:
    stats, stats_df = calculate_statistics(data, name)
    if stats:
        all_stats.append(stats)
        print(f"\n{'='*50}")
        print(f"Statistics for {name}:")
        print(f"{'='*50}")
        for key, value in stats.items():
            if key != 'Dataset':
                print(f"{key:20s}: {value}")
        print(f"\nFeature Statistics:")
        print(stats_df.head())

## 4. Time Series Visualization

In [ ]:
def plot_time_series_sample(data, labels=None, name="Dataset", n_samples=3, n_features=5):
    """Plot sample time series from the dataset"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    # Select random samples
    n_samples = min(n_samples, len(data))
    sample_indices = np.random.choice(len(data), n_samples, replace=False)
    
    # Select features to plot
    n_features = min(n_features, data.shape[-1])
    
    fig, axes = plt.subplots(n_samples, n_features, figsize=(20, 3*n_samples))
    if n_samples == 1:
        axes = axes.reshape(1, -1)
    if n_features == 1:
        axes = axes.reshape(-1, 1)
    
    for i, idx in enumerate(sample_indices):
        sample = data[idx]
        
        for j in range(n_features):
            axes[i, j].plot(sample[:, j], linewidth=0.8)
            
            # Highlight anomalies if labels are provided
            if labels is not None and idx < len(labels):
                anomaly_mask = labels[idx].astype(bool)
                if anomaly_mask.any():
                    axes[i, j].scatter(np.where(anomaly_mask)[0], 
                                     sample[anomaly_mask, j],
                                     color='red', s=10, alpha=0.6,
                                     label='Anomaly')
            
            axes[i, j].set_title(f'Sample {idx} - Feature {j}')
            axes[i, j].grid(True, alpha=0.3)
            
            if j == 0:
                axes[i, j].set_ylabel('Value')
            if i == n_samples - 1:
                axes[i, j].set_xlabel('Time Step')
    
    plt.suptitle(f'{name} - Time Series Samples', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

# Plot samples from each dataset
plot_time_series_sample(smap_train, None, "SMAP Training", n_samples=2, n_features=4)
plot_time_series_sample(smap_test, smap_labels, "SMAP Test (with anomalies)", n_samples=2, n_features=4)
plot_time_series_sample(msl_train, None, "MSL Training", n_samples=2, n_features=4)
plot_time_series_sample(msl_test, msl_labels, "MSL Test (with anomalies)", n_samples=2, n_features=4)

## 5. Interactive Time Series Exploration

In [ ]:
def create_interactive_plot(data, labels=None, name="Dataset", sample_idx=0):
    """Create interactive Plotly visualization"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    sample = data[sample_idx]
    n_features = sample.shape[1]
    
    # Create subplots
    fig = make_subplots(
        rows=n_features, cols=1,
        subplot_titles=[f'Feature {i}' for i in range(n_features)],
        vertical_spacing=0.05
    )
    
    for i in range(n_features):
        # Add time series
        fig.add_trace(
            go.Scatter(x=list(range(len(sample))),
                      y=sample[:, i],
                      mode='lines',
                      name=f'Feature {i}',
                      line=dict(width=1)),
            row=i+1, col=1
        )
        
        # Add anomalies if available
        if labels is not None and sample_idx < len(labels):
            anomaly_mask = labels[sample_idx].astype(bool)
            if anomaly_mask.any():
                fig.add_trace(
                    go.Scatter(x=np.where(anomaly_mask)[0],
                              y=sample[anomaly_mask, i],
                              mode='markers',
                              name=f'Anomaly F{i}',
                              marker=dict(color='red', size=5)),
                    row=i+1, col=1
                )
    
    fig.update_layout(
        height=200*n_features,
        title_text=f"{name} - Sample {sample_idx} Interactive View",
        showlegend=False,
        hovermode='x unified'
    )
    
    fig.show()

# Create interactive plots
if smap_test is not None:
    create_interactive_plot(smap_test, smap_labels, "SMAP Test", sample_idx=0)
if msl_test is not None:
    create_interactive_plot(msl_test, msl_labels, "MSL Test", sample_idx=0)

## 6. Distribution Analysis

In [ ]:
def plot_distributions(data, name="Dataset", n_features=8):
    """Plot distribution of values for each feature"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    # Reshape data to 2D
    if len(data.shape) == 3:
        data_2d = data.reshape(-1, data.shape[-1])
    else:
        data_2d = data
    
    n_features = min(n_features, data_2d.shape[1])
    
    fig, axes = plt.subplots(2, n_features//2, figsize=(15, 8))
    axes = axes.flatten()
    
    for i in range(n_features):
        feature_data = data_2d[:, i]
        
        # Remove NaN values
        feature_data = feature_data[~np.isnan(feature_data)]
        
        # Plot histogram with KDE
        axes[i].hist(feature_data, bins=50, alpha=0.7, density=True, edgecolor='black')
        
        # Fit and plot normal distribution
        mu, std = stats.norm.fit(feature_data)
        xmin, xmax = axes[i].get_xlim()
        x = np.linspace(xmin, xmax, 100)
        p = stats.norm.pdf(x, mu, std)
        axes[i].plot(x, p, 'r-', linewidth=2, label=f'Normal\nμ={mu:.2f}\nσ={std:.2f}')
        
        axes[i].set_title(f'Feature {i} Distribution')
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Density')
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)
    
    plt.suptitle(f'{name} - Feature Distributions', fontsize=16)
    plt.tight_layout()
    plt.show()

# Plot distributions
plot_distributions(smap_train, "SMAP Training")
plot_distributions(msl_train, "MSL Training")

## 7. Anomaly Analysis

In [ ]:
def analyze_anomalies(test_data, labels, name="Dataset"):
    """Analyze anomaly patterns in the dataset"""
    if test_data is None or labels is None:
        print(f"No data or labels available for {name}")
        return
    
    # Calculate anomaly statistics
    total_points = np.prod(labels.shape)
    anomaly_points = labels.sum()
    anomaly_percentage = (anomaly_points / total_points) * 100
    
    print(f"\n{'='*50}")
    print(f"Anomaly Analysis for {name}")
    print(f"{'='*50}")
    print(f"Total data points: {total_points:,}")
    print(f"Anomaly points: {anomaly_points:,}")
    print(f"Anomaly percentage: {anomaly_percentage:.2f}%")
    
    # Analyze anomaly lengths
    anomaly_lengths = []
    for i in range(len(labels)):
        anomaly_mask = labels[i].astype(bool)
        if anomaly_mask.any():
            # Find consecutive anomalies
            diff = np.diff(np.concatenate(([0], anomaly_mask.astype(int), [0])))
            starts = np.where(diff == 1)[0]
            ends = np.where(diff == -1)[0]
            lengths = ends - starts
            anomaly_lengths.extend(lengths)
    
    if anomaly_lengths:
        print(f"\nAnomaly Sequence Statistics:")
        print(f"  Number of sequences: {len(anomaly_lengths)}")
        print(f"  Mean length: {np.mean(anomaly_lengths):.2f} time steps")
        print(f"  Median length: {np.median(anomaly_lengths):.0f} time steps")
        print(f"  Min length: {np.min(anomaly_lengths)} time steps")
        print(f"  Max length: {np.max(anomaly_lengths)} time steps")
        
        # Plot anomaly length distribution
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.hist(anomaly_lengths, bins=30, edgecolor='black', alpha=0.7)
        plt.xlabel('Anomaly Length (time steps)')
        plt.ylabel('Frequency')
        plt.title(f'{name} - Anomaly Length Distribution')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.boxplot(anomaly_lengths)
        plt.ylabel('Anomaly Length (time steps)')
        plt.title(f'{name} - Anomaly Length Boxplot')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return anomaly_lengths

# Analyze anomalies
smap_anomaly_lengths = analyze_anomalies(smap_test, smap_labels, "SMAP")
msl_anomaly_lengths = analyze_anomalies(msl_test, msl_labels, "MSL")

## 8. Correlation Analysis

In [ ]:
def plot_correlation_matrix(data, name="Dataset", max_features=20):
    """Plot correlation matrix between features"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    # Reshape to 2D if necessary
    if len(data.shape) == 3:
        data_2d = data.reshape(-1, data.shape[-1])
    else:
        data_2d = data
    
    # Limit features for visualization
    n_features = min(max_features, data_2d.shape[1])
    data_subset = data_2d[:, :n_features]
    
    # Calculate correlation matrix
    corr_matrix = np.corrcoef(data_subset.T)
    
    # Create heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, 
                annot=n_features <= 10, 
                fmt='.2f',
                cmap='coolwarm',
                center=0,
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})
    
    plt.title(f'{name} - Feature Correlation Matrix', fontsize=16)
    plt.xlabel('Feature Index')
    plt.ylabel('Feature Index')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated features
    high_corr_pairs = []
    for i in range(n_features):
        for j in range(i+1, n_features):
            if abs(corr_matrix[i, j]) > 0.8:
                high_corr_pairs.append((i, j, corr_matrix[i, j]))
    
    if high_corr_pairs:
        print(f"\nHighly correlated feature pairs (|correlation| > 0.8):")
        for i, j, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True)[:10]:
            print(f"  Feature {i} - Feature {j}: {corr:.3f}")
    
    return corr_matrix

# Plot correlation matrices
smap_corr = plot_correlation_matrix(smap_train, "SMAP Training")
msl_corr = plot_correlation_matrix(msl_train, "MSL Training")

## 9. Time Series Decomposition

In [ ]:
def decompose_time_series(data, name="Dataset", sample_idx=0, feature_idx=0):
    """Decompose time series into trend, seasonal, and residual components"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    # Get single time series
    ts = data[sample_idx][:, feature_idx]
    
    # Create pandas series with datetime index
    dates = pd.date_range(start='2023-01-01', periods=len(ts), freq='H')
    ts_series = pd.Series(ts, index=dates)
    
    try:
        # Perform decomposition
        decomposition = seasonal_decompose(ts_series, model='additive', period=24)
        
        # Plot decomposition
        fig, axes = plt.subplots(4, 1, figsize=(15, 10))
        
        ts_series.plot(ax=axes[0])
        axes[0].set_title(f'{name} - Original Time Series (Sample {sample_idx}, Feature {feature_idx})')
        axes[0].set_ylabel('Value')
        
        decomposition.trend.plot(ax=axes[1])
        axes[1].set_title('Trend Component')
        axes[1].set_ylabel('Trend')
        
        decomposition.seasonal.plot(ax=axes[2])
        axes[2].set_title('Seasonal Component')
        axes[2].set_ylabel('Seasonal')
        
        decomposition.resid.plot(ax=axes[3])
        axes[3].set_title('Residual Component')
        axes[3].set_ylabel('Residual')
        axes[3].set_xlabel('Time')
        
        plt.tight_layout()
        plt.show()
        
        return decomposition
    except Exception as e:
        print(f"Could not decompose time series: {e}")
        return None

# Decompose sample time series
if smap_train is not None and len(smap_train) > 0:
    smap_decomp = decompose_time_series(smap_train, "SMAP", sample_idx=0, feature_idx=0)
if msl_train is not None and len(msl_train) > 0:
    msl_decomp = decompose_time_series(msl_train, "MSL", sample_idx=0, feature_idx=0)

## 10. Autocorrelation Analysis

In [ ]:
def analyze_autocorrelation(data, name="Dataset", sample_idx=0, n_features=4):
    """Analyze autocorrelation and partial autocorrelation"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    n_features = min(n_features, data.shape[-1])
    
    fig, axes = plt.subplots(n_features, 2, figsize=(15, 3*n_features))
    if n_features == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n_features):
        ts = data[sample_idx][:, i]
        
        # ACF
        plot_acf(ts, lags=40, ax=axes[i, 0])
        axes[i, 0].set_title(f'Feature {i} - Autocorrelation')
        
        # PACF
        try:
            plot_pacf(ts, lags=40, ax=axes[i, 1])
            axes[i, 1].set_title(f'Feature {i} - Partial Autocorrelation')
        except:
            axes[i, 1].text(0.5, 0.5, 'Could not compute PACF', 
                           ha='center', va='center')
    
    plt.suptitle(f'{name} - ACF and PACF Analysis (Sample {sample_idx})', fontsize=16)
    plt.tight_layout()
    plt.show()

# Analyze autocorrelation
if smap_train is not None and len(smap_train) > 0:
    analyze_autocorrelation(smap_train, "SMAP", sample_idx=0)
if msl_train is not None and len(msl_train) > 0:
    analyze_autocorrelation(msl_train, "MSL", sample_idx=0)

## 11. Stationarity Testing

In [ ]:
def test_stationarity(data, name="Dataset", n_samples=10, n_features=5):
    """Test stationarity using Augmented Dickey-Fuller test"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    n_samples = min(n_samples, len(data))
    n_features = min(n_features, data.shape[-1])
    
    results = []
    
    for i in range(n_samples):
        for j in range(n_features):
            ts = data[i][:, j]
            
            # Remove NaN values
            ts = ts[~np.isnan(ts)]
            
            if len(ts) > 10:  # Need sufficient data for test
                try:
                    adf_result = adfuller(ts, autolag='AIC')
                    results.append({
                        'Sample': i,
                        'Feature': j,
                        'ADF Statistic': adf_result[0],
                        'p-value': adf_result[1],
                        'Critical Value (5%)': adf_result[4]['5%'],
                        'Stationary': adf_result[1] < 0.05
                    })
                except:
                    pass
    
    if results:
        results_df = pd.DataFrame(results)
        
        print(f"\n{'='*50}")
        print(f"Stationarity Test Results for {name}")
        print(f"{'='*50}")
        print(f"Tested {len(results)} time series")
        print(f"Stationary series: {results_df['Stationary'].sum()} ({results_df['Stationary'].mean()*100:.1f}%)")
        print(f"\nSample results:")
        print(results_df.head(10))
        
        # Visualize results
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        # p-value distribution
        axes[0].hist(results_df['p-value'], bins=30, edgecolor='black', alpha=0.7)
        axes[0].axvline(x=0.05, color='red', linestyle='--', label='Significance level (0.05)')
        axes[0].set_xlabel('p-value')
        axes[0].set_ylabel('Frequency')
        axes[0].set_title(f'{name} - p-value Distribution')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Stationarity by feature
        feature_stats = results_df.groupby('Feature')['Stationary'].mean()
        axes[1].bar(feature_stats.index, feature_stats.values, edgecolor='black', alpha=0.7)
        axes[1].set_xlabel('Feature Index')
        axes[1].set_ylabel('Proportion Stationary')
        axes[1].set_title(f'{name} - Stationarity by Feature')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        return results_df
    
    return None

# Test stationarity
smap_stationarity = test_stationarity(smap_train, "SMAP")
msl_stationarity = test_stationarity(msl_train, "MSL")

## 12. Frequency Domain Analysis

In [ ]:
def analyze_frequency_domain(data, name="Dataset", sample_idx=0, n_features=4):
    """Analyze time series in frequency domain using FFT"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    n_features = min(n_features, data.shape[-1])
    
    fig, axes = plt.subplots(n_features, 2, figsize=(15, 3*n_features))
    if n_features == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n_features):
        ts = data[sample_idx][:, i]
        
        # Remove NaN values
        ts = ts[~np.isnan(ts)]
        
        # Time domain
        axes[i, 0].plot(ts)
        axes[i, 0].set_title(f'Feature {i} - Time Domain')
        axes[i, 0].set_xlabel('Time Step')
        axes[i, 0].set_ylabel('Value')
        axes[i, 0].grid(True, alpha=0.3)
        
        # Frequency domain
        n = len(ts)
        yf = fft(ts)
        xf = fftfreq(n, 1)[:n//2]
        
        axes[i, 1].plot(xf, 2.0/n * np.abs(yf[:n//2]))
        axes[i, 1].set_title(f'Feature {i} - Frequency Domain')
        axes[i, 1].set_xlabel('Frequency')
        axes[i, 1].set_ylabel('Amplitude')
        axes[i, 1].grid(True, alpha=0.3)
        
        # Mark dominant frequencies
        power = np.abs(yf[:n//2])
        peaks, _ = find_peaks(power, height=np.percentile(power, 90))
        if len(peaks) > 0:
            axes[i, 1].plot(xf[peaks], 2.0/n * power[peaks], 'ro', markersize=5)
    
    plt.suptitle(f'{name} - Frequency Domain Analysis (Sample {sample_idx})', fontsize=16)
    plt.tight_layout()
    plt.show()

# Analyze frequency domain
if smap_train is not None and len(smap_train) > 0:
    analyze_frequency_domain(smap_train, "SMAP", sample_idx=0)
if msl_train is not None and len(msl_train) > 0:
    analyze_frequency_domain(msl_train, "MSL", sample_idx=0)

## 13. Dimensionality Reduction

In [ ]:
def perform_dimensionality_reduction(data, labels=None, name="Dataset", n_samples=1000):
    """Perform PCA and t-SNE for visualization"""
    if data is None:
        print(f"No data available for {name}")
        return
    
    # Reshape to 2D
    if len(data.shape) == 3:
        data_2d = data.reshape(data.shape[0], -1)
    else:
        data_2d = data
    
    # Sample data if too large
    if len(data_2d) > n_samples:
        indices = np.random.choice(len(data_2d), n_samples, replace=False)
        data_sample = data_2d[indices]
        labels_sample = labels[indices] if labels is not None else None
    else:
        data_sample = data_2d
        labels_sample = labels
    
    # Standardize data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_sample)
    
    # PCA
    pca = PCA(n_components=min(50, data_scaled.shape[1]))
    pca_result = pca.fit_transform(data_scaled)
    
    # t-SNE (on PCA result for speed)
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    tsne_result = tsne.fit_transform(pca_result[:, :10])
    
    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # PCA variance explained
    axes[0, 0].plot(np.cumsum(pca.explained_variance_ratio_))
    axes[0, 0].set_xlabel('Number of Components')
    axes[0, 0].set_ylabel('Cumulative Explained Variance')
    axes[0, 0].set_title('PCA - Variance Explained')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].axhline(y=0.95, color='r', linestyle='--', label='95% variance')
    axes[0, 0].legend()
    
    # PCA 2D projection
    scatter1 = axes[0, 1].scatter(pca_result[:, 0], pca_result[:, 1], 
                                  c=labels_sample.mean(axis=1) if labels_sample is not None else None,
                                  cmap='coolwarm', alpha=0.6, s=10)
    axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    axes[0, 1].set_title('PCA - 2D Projection')
    axes[0, 1].grid(True, alpha=0.3)
    if labels_sample is not None:
        plt.colorbar(scatter1, ax=axes[0, 1], label='Anomaly Score')
    
    # t-SNE projection
    scatter2 = axes[1, 0].scatter(tsne_result[:, 0], tsne_result[:, 1],
                                  c=labels_sample.mean(axis=1) if labels_sample is not None else None,
                                  cmap='coolwarm', alpha=0.6, s=10)
    axes[1, 0].set_xlabel('t-SNE 1')
    axes[1, 0].set_ylabel('t-SNE 2')
    axes[1, 0].set_title('t-SNE - 2D Projection')
    axes[1, 0].grid(True, alpha=0.3)
    if labels_sample is not None:
        plt.colorbar(scatter2, ax=axes[1, 0], label='Anomaly Score')
    
    # Feature importance (based on PCA components)
    feature_importance = np.abs(pca.components_[0])
    top_features = np.argsort(feature_importance)[-20:]
    axes[1, 1].barh(range(len(top_features)), feature_importance[top_features])
    axes[1, 1].set_xlabel('Absolute Loading')
    axes[1, 1].set_ylabel('Feature Rank')
    axes[1, 1].set_title('Top 20 Features - PC1 Loadings')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle(f'{name} - Dimensionality Reduction Analysis', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return pca, tsne_result

# Perform dimensionality reduction
if smap_test is not None:
    smap_pca, smap_tsne = perform_dimensionality_reduction(smap_test, smap_labels, "SMAP")
if msl_test is not None:
    msl_pca, msl_tsne = perform_dimensionality_reduction(msl_test, msl_labels, "MSL")

## 14. Summary and Insights

In [ ]:
def generate_summary():
    """Generate summary of key findings"""
    
    print("\n" + "="*60)
    print("DATA EXPLORATION SUMMARY")
    print("="*60)
    
    print("\n📊 DATASET CHARACTERISTICS:")
    print("-" * 40)
    
    datasets_info = [
        ("SMAP Train", smap_train),
        ("SMAP Test", smap_test),
        ("MSL Train", msl_train),
        ("MSL Test", msl_test)
    ]
    
    for name, data in datasets_info:
        if data is not None:
            print(f"\n{name}:")
            print(f"  • Shape: {data.shape}")
            print(f"  • Memory: {data.nbytes / (1024**2):.2f} MB")
            print(f"  • Value range: [{np.min(data):.3f}, {np.max(data):.3f}]")
            print(f"  • Missing values: {np.isnan(data).sum()} ({np.isnan(data).mean()*100:.2f}%)")
    
    print("\n🔍 KEY OBSERVATIONS:")
    print("-" * 40)
    
    observations = [
        "1. Time Series Properties:",
        "   • Both datasets contain multivariate time series from IoT sensors",
        "   • Data shows complex temporal patterns with varying frequencies",
        "   • Some features exhibit strong autocorrelation",
        "",
        "2. Anomaly Characteristics:",
        "   • Anomalies appear as both point outliers and sequential patterns",
        "   • Anomaly duration varies significantly (from single points to extended sequences)",
        "   • Different features may have correlated anomalies",
        "",
        "3. Statistical Properties:",
        "   • Features have different scales and distributions",
        "   • Some features show non-stationary behavior",
        "   • Correlation exists between certain sensor readings",
        "",
        "4. Data Quality:",
        "   • Generally clean data with minimal missing values",
        "   • Some features may benefit from normalization",
        "   • Outliers present in normal operation (not just anomalies)"
    ]
    
    for obs in observations:
        print(obs)
    
    print("\n💡 RECOMMENDATIONS FOR MODELING:")
    print("-" * 40)
    
    recommendations = [
        "1. Preprocessing:",
        "   ✓ Apply normalization (MinMax or Standard scaling)",
        "   ✓ Handle missing values with interpolation",
        "   ✓ Consider feature engineering (rolling statistics, FFT features)",
        "",
        "2. Model Selection:",
        "   ✓ LSTM-based models suitable for capturing temporal dependencies",
        "   ✓ Autoencoder architectures for reconstruction-based detection",
        "   ✓ Consider ensemble approaches for robustness",
        "",
        "3. Training Strategy:",
        "   ✓ Use sliding windows for sequence generation",
        "   ✓ Implement proper train/validation split",
        "   ✓ Monitor for overfitting on normal patterns",
        "",
        "4. Evaluation:",
        "   ✓ Focus on precision-recall trade-offs",
        "   ✓ Consider point-wise and range-based metrics",
        "   ✓ Validate on both datasets for generalization"
    ]
    
    for rec in recommendations:
        print(rec)
    
    print("\n" + "="*60)
    print("END OF EXPLORATION")
    print("="*60)

# Generate summary
generate_summary()

## 15. Export Key Findings

In [ ]:
# Save key statistics and findings
import json
from datetime import datetime

exploration_results = {
    'timestamp': datetime.now().isoformat(),
    'datasets': {},
    'anomaly_analysis': {},
    'recommendations': []
}

# Collect dataset info
for name, data, labels in [('smap_train', smap_train, None),
                           ('smap_test', smap_test, smap_labels),
                           ('msl_train', msl_train, None),
                           ('msl_test', msl_test, msl_labels)]:
    if data is not None:
        exploration_results['datasets'][name] = {
            'shape': data.shape,
            'memory_mb': float(data.nbytes / (1024**2)),
            'min_value': float(np.min(data)),
            'max_value': float(np.max(data)),
            'mean_value': float(np.mean(data)),
            'std_value': float(np.std(data)),
            'missing_count': int(np.isnan(data).sum()),
            'has_anomalies': labels is not None
        }

# Save to JSON
output_path = '../data/processed/data_exploration_results.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(exploration_results, f, indent=2)

print(f"\n✅ Exploration results saved to: {output_path}")
print("\n📝 Next Steps:")
print("   1. Run model_experiments.ipynb for model development")
print("   2. Use visualization.ipynb for detailed result analysis")
print("   3. Deploy models using the main application scripts")